# CalEITC

California's EITC applies the federal EITC rules, with some modified parameters.

## Amounts

The chart below shows the CalEITC value for filers with between zero and four children.

In [6]:
from policyengine_us import Simulation


def create_simulation(count_children):
    situation = {
        "people": {
            "person": {
                "age": 19,
            },
        },
        "tax_units": {
            "tax_unit": {
                "members": ["person"],
            },
        },
        "households": {
            "household": {
                "members": ["person"],
                "state_code": {2023: "CA"},
            },
        },
        "axes": [
            [
                {
                    "name": "employment_income",
                    "min": 0,
                    "max": 30_000,
                    "count": 500,
                }
            ]
        ],
    }
    for child in range(count_children):
        situation["people"][f"child_{child}"] = {"age": {2023: 5}}
        situation["tax_units"]["tax_unit"]["members"].append(f"child_{child}")
        situation["households"]["household"]["members"].append(
            f"child_{child}"
        )

    return Simulation(situation=situation)


import plotly.express as px
import pandas as pd

LIGHT_GRAY = "#F5F5F5"
GRAY = "#BDBDBD"
BLUE = "#5091cc"
LIGHT_BLUE = "lightblue"
DARK_BLUE = "darkblue"

COLOR_MAP = {"0": GRAY, "1": LIGHT_BLUE, "2": BLUE, "3": DARK_BLUE}

simulations = {i: create_simulation(i) for i in range(4)}

df = pd.DataFrame()
df["Employment income"] = simulations[0].calculate("employment_income", 2023)
for i in range(4):
    df[f"{i}"] = simulations[i].calculate("ca_eitc", 2023)

px.line(
    df,
    x="Employment income",
    y=df.columns[1:],
    color_discrete_map=COLOR_MAP,
).update_layout(
    title="CalEITC by child count",
    xaxis_title="Employment income",
    xaxis_tickformat="$,.0f",
    yaxis_title="CalEITC",
    yaxis_tickformat="$,.0f",
    legend_title="Child count",
    height=600,
    width=800,
    template="plotly_white",
).show()

## Marginal tax rates

CalEITC imposes both positive and negative marginal tax rates, depending on the taxpayer's income.

In [8]:
df = pd.DataFrame()
df["Employment income"] = simulations[0].calculate("employment_income", 2023)[
    :-1
]
for i in range(4):
    earnings = simulations[i].calculate(
        "employment_income", 2023, map_to="tax_unit"
    )
    ca_eitc = simulations[i].calculate("ca_eitc", 2023)
    mtr = 1 - (ca_eitc[1:] - ca_eitc[:-1]) / (earnings[1:] - earnings[:-1])
    df[f"{i}"] = mtr

px.line(
    df,
    x="Employment income",
    y=df.columns[1:],
    color_discrete_map=COLOR_MAP,
).update_layout(
    title="CalEITC MTR by child count",
    xaxis_title="Employment income",
    xaxis_tickformat="$,.0f",
    yaxis_title="MTR",
    yaxis_tickformat=".1%",
    legend_title="Child count",
    height=600,
    width=800,
    template="plotly_white",
).show()